# 1.문제 정의

In [5]:
import numpy as np
import pandas as pd

# 데이터 불러오기
data = pd.read_csv('/content/drive/MyDrive/2023-2알고리즘/[Project] 주어진 행렬 공간 중에서 최대 volume 구하기/input.csv')
vectors = data.iloc[0:20].to_numpy()  # 첫 행은 인덱스, 나머지가 벡터 값

# 초기 설정
num_vectors = 20    # 행렬의 행 수
population_size = 100  # 인구 크기
vector_length = 10000  # 전체 벡터의 수 설정 (예시로 10000으로 가정)
num_generations = 100  # 세대 수
mutation_rate = 0.1   # 돌연변이율
print(vectors.shape)

(20, 10000)


# 2. 초기 인구 생성


In [11]:
def initialize_population(pop_size, num_vectors):
    # 각 행렬의 열이 중복되지 않도록 선택
    population = []
    for _ in range(pop_size):
        x_indices = np.random.choice(num_vectors, 2, replace=False)
        y_indices = np.random.choice(vector_length, 2, replace=False)
        indices = np.stack((x_indices, y_indices), axis=-1)
        population.append(indices)
    return np.array(population)

# x와 y 좌표를 결합하여 2차원 인덱스 생성
population = initialize_population(population_size, num_vectors)
print(population)
population = population.reshape(100,4)
print(population)

[[[   9 3777]
  [  15 5547]]

 [[  11 8059]
  [  12 2687]]

 [[  15 3815]
  [  13 7404]]

 [[  13 3513]
  [  15 8977]]

 [[  15 1602]
  [   2 8168]]

 [[  17 2408]
  [  19 6029]]

 [[  10 6921]
  [  16 4378]]

 [[   0 2238]
  [  19 4766]]

 [[  18 8572]
  [  19 3637]]

 [[   1 6763]
  [   6 4115]]

 [[   0  369]
  [   1  913]]

 [[  15 7980]
  [  18 6394]]

 [[   1 6679]
  [   8 9658]]

 [[   6 1166]
  [  17 8287]]

 [[   1 3425]
  [   0 4132]]

 [[   3 9046]
  [  17  553]]

 [[   2 1858]
  [  12 6053]]

 [[  13  432]
  [   9 2833]]

 [[   0 6708]
  [  11 5966]]

 [[  17  775]
  [  18 2008]]

 [[  16 9016]
  [   9 7544]]

 [[  10  846]
  [  15 5913]]

 [[   6 6251]
  [  16 5035]]

 [[   2 2069]
  [  19 6135]]

 [[  11 1491]
  [  12 7923]]

 [[  17 1395]
  [   0 2347]]

 [[   8 1427]
  [   6 8592]]

 [[  12 2552]
  [   2 7861]]

 [[   4  275]
  [   7 1347]]

 [[  11 3347]
  [  19 8884]]

 [[  17 9162]
  [   3 5887]]

 [[   7 6946]
  [   5 7656]]

 [[  15 5029]
  [   9 1518]]

 [[   8 66

# 3. 적합도 함수 정의


In [12]:
# 3. 적합도 함수 정의
def fitness(matrix):
    matrix_product = np.dot(matrix, matrix.transpose())
    matrix_det = np.linalg.det(matrix_product)
    if np.abs(matrix_det) < 1e-5:  # 행렬식이 매우 작은 경우 낮은 적합도 반환
        return 0
    volume = np.sqrt(np.abs(matrix_det))
    return volume

# print(fitness(vectors))

def evaluate_population(vectors):
    fitness_scores = []
    # population (100, 100, 20, 2) 100개의 개체군이 각각(100x20) 행렬로 이루어져 있으며,
    # 각 개체군의 요소는 vectors의 index [x, y]쌍으로 가지고 있다.
    matrix = []
    for individual in population:
      # 각 개체군을 가져옴. mtx.shape = (100, 20, 20)
      # 각 개체의 인덱스를 사용하여 vectors에서 실제 값을 가져와 행렬을 생성
      matrix.append([vectors[x1:x2][y1:y2] for x1, x2, y1, y2 in individual])

    matrix = np.array(matrix)
      # print("matrix")
      # print(matrix)
      # print(fitness(matrix))
    fitness_scores.append(fitness(matrix))
    return np.array(fitness_scores)



fitness_scores = evaluate_population(vectors)
print(fitness_scores.shape)
print(*fitness_scores, sep = '\n')

TypeError: ignored

# 4. 선택

In [ ]:
def select_parents(population, fitness_scores):
    # fitness_scores.sum()이 0인 경우 예외 처리
    if fitness_scores.sum() == 0:
        # 확률을 균등하게 설정
        probabilities = np.ones(len(fitness_scores)) / len(fitness_scores)
    else:
        probabilities = fitness_scores / fitness_scores.sum()

    selected_indices = np.random.choice(len(population), size=len(population), replace=True, p=probabilities)
    return population[selected_indices]

parents = select_parents(population, fitness_scores)

# 5. 교차

In [ ]:
def crossover(parents):
    offspring = []
    for i in range(0, len(parents), 2):
        parent1 = parents[i]
        parent2 = parents[i + 1]
        cut = np.random.randint(1, len(parent1)-1)
        child1 = np.concatenate([parent1[:cut], parent2[cut:]])
        child2 = np.concatenate([parent2[:cut], parent1[cut:]])
        offspring.extend([child1, child2])
    return np.array(offspring)

offspring = crossover(parents)


# 6. 돌연변이

In [ ]:
# 돌연변이 함수
def mutation(offspring_crossover, mutation_rate, num_vectors):
    for idx in range(offspring_crossover.shape[0]):
        if np.random.rand() < mutation_rate:
            random_index = np.random.randint(offspring_crossover.shape[1])
            random_value = np.random.randint(num_vectors)
            offspring_crossover[idx, random_index] = random_value
    return offspring_crossover
# 돌연변이 함수 호출
num_vectors = 10000  # 전체 벡터의 수 설정
mutated_offspring = mutation(offspring, mutation_rate, num_vectors)


# 7-8. 새로운 세대 생성 및 종료 조건 검사

In [ ]:
for generation in range(num_generations):
    fitness_scores = evaluate_population(population, vectors)
    parents = select_parents(population, fitness_scores)
    offspring = crossover(parents)  # 여기서 두 번째 인자 제거
    mutated_offspring = mutation(offspring, mutation_rate, num_vectors)
    population = mutated_offspring
    # 종료 조건 검사
    # ...


TypeError: ignored

# 9. 결과 추출

In [ ]:
final_fitness = evaluate_population(population, vectors)
best_index = np.argmax(final_fitness)
best_solution = population[best_index]
best_volume = final_fitness[best_index]

print("최대 볼륨:", best_volume)
print("해당 벡터의 인덱스:", best_solution)


TypeError: ignored